In [1]:
!pip install ultralytics flask pyngrok opencv-python-headless
!apt install ffmpeg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 896.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
html_code = """
<!DOCTYPE html>
<html lang="he" dir="rtl">
<head>
    <meta charset="utf-8">
    <title>People Counter - AI Developments</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 40px; background: #fafbfc; }
        .container { display: flex; flex-direction: row; justify-content: center; align-items: flex-start; gap: 40px; }
        .video-box { flex: 2; }
        .params-box {
            flex: 1;
            background: #fff;
            border-radius: 12px;
            box-shadow: 0 2px 8px #0001;
            padding: 32px 24px;
            min-width: 320px;
            max-width: 400px;
        }
        .params-box h3 { margin-top: 0; }
        label { font-weight: bold; }
        .param-group { margin-bottom: 20px; }
        .param-group input[type="text"], .param-group input[type="number"] { width: 100%; padding: 5px; border: 1px solid #bbb; border-radius: 5px; }
        .param-group input[type="range"] { width: 85%; }
        .param-group input[type="checkbox"] { margin-left: 8px; }
        .save-btn {
            background: #1b83e3;
            color: white;
            border: none;
            border-radius: 7px;
            padding: 10px 30px;
            font-size: 1.2em;
            cursor: pointer;
        }
        .save-btn:hover { background: #1568b2; }
        #stats-box {
            margin-top: 40px;
            background: #f4f6f9;
            border-radius: 12px;
            box-shadow: 0 2px 8px #0001;
            padding: 24px;
            direction: rtl;
        }
        #stats-box hr { margin: 16px 0; }
        #historyChart { background: #fff; border-radius: 8px; }
    </style>
</head>
<body>
    <h1>AI Developments - מונה אנשים בבינה מלאכותית</h1>
    <div class="container">
        <!-- חלק שמאל: וידאו -->
        <div class="video-box">
            <img id="videoStream" src="/video_feed" width="850" style="border-radius: 12px; box-shadow: 0 4px 24px #0002;">
            <!-- סטטיסטיקות מתעדכנות -->
            <div id="stats-box">
                <h3>סטטיסטיקות בזמן אמת (10 דקות אחרונות)</h3>
                <div>
                    <strong>ממוצע אנשים:</strong> <span id="avg_people">-</span><br>
                    <strong>ממוצע רכבים:</strong> <span id="avg_cars">-</span><br>
                    <strong>ממוצע משאיות:</strong> <span id="avg_trucks">-</span>
                </div>
                <hr>
                <h3>גרף שעה אחרונה</h3>
                <canvas id="historyChart" height="130"></canvas>
            </div>
        </div>
        <!-- חלק ימין: פרמטרים -->
        <div class="params-box">
            <form method="post">
                <h3>הגדרות זיהוי</h3>
                <!-- כתובת YouTube -->
                <div class="param-group">
                    <label for="youtube_url">כתובת YouTube Live:</label><br>
                    <input type="text" id="youtube_url" name="youtube_url" value="{{ youtube_url|e }}">
                </div>
                <!-- Drop down -->
                <div class="param-group">
                    <label for="predefined_stream">או בחר ממצלמות לייב פופולריות:</label><br>
                    <select id="predefined_stream">
                        <option value="">-- בחר ממצלמות --</option>
                        {% for stream in predefined_streams %}
                            <option value="{{ stream.url }}">{{ stream.name }}</option>
                        {% endfor %}
                    </select>
                </div>
                <!-- סליידר קונפידנס -->
                <div class="param-group">
                    <label for="conf">סף זיהוי (Confidence):</label><br>
                    <input type="range" id="conf" name="conf" min="0.01" max="1" step="0.01" value="{{ conf }}">
                    <span id="confValue">{{ conf }}</span>
                </div>
                <!-- צ'קבוקסים לאובייקטים -->
                <div class="param-group">
                    <label>מה לזהות?</label><br>
                    <input type="checkbox" id="detect_people" name="detect_people" {% if detect_people %}checked{% endif %}>
                    <label for="detect_people">אנשים</label>
                    <input type="checkbox" id="detect_cars" name="detect_cars" {% if detect_cars %}checked{% endif %}>
                    <label for="detect_cars">מכוניות</label>
                    <input type="checkbox" id="detect_trucks" name="detect_trucks" {% if detect_trucks %}checked{% endif %}>
                    <label for="detect_trucks">משאיות</label>
                </div>
                <!-- סליידר פריימים בשנייה -->
                <div class="param-group">
                    <label for="fps">פריימים לשנייה:</label><br>
                    <input type="range" id="fps" name="fps" min="1" max="10" step="1" value="{{ fps }}">
                    <span id="fpsValue">{{ fps }}</span>
                </div>
                <div class="param-group" style="text-align: left;">
                    <button type="submit" class="save-btn">שמור</button>
                </div>
            </form>
        </div>
    </div>
    <!-- צ'ארט גרף -->
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <script>
        // עדכון ערך קונפידנס לייד הסליידר
        document.getElementById('conf').oninput = function() {
            document.getElementById('confValue').innerText = this.value;
        };
        document.getElementById('fps').oninput = function() {
            document.getElementById('fpsValue').innerText = this.value;
        };
        // כאשר נבחר סטרים דרופדאון - מלא אוטומטית באינפוט
        document.getElementById('predefined_stream').onchange = function() {
            if (this.value) {
                document.getElementById('youtube_url').value = this.value;
            }
        };
        // סטטיסטיקות וגרף
        let historyChart = null;

        function updateStats() {
            fetch('/stats')
            .then(r => r.json())
            .then(data => {
                document.getElementById('avg_people').innerText = data.avg_people_10min;
                document.getElementById('avg_cars').innerText = data.avg_cars_10min;
                document.getElementById('avg_trucks').innerText = data.avg_trucks_10min;

                const labels = data.chart.map(pt => {
                    const date = new Date(pt.t * 1000);
                    return date.getHours().toString().padStart(2, '0') + ':' + date.getMinutes().toString().padStart(2, '0');
                });
                const peopleData = data.chart.map(pt => pt.people);
                const carsData = data.chart.map(pt => pt.cars);
                const trucksData = data.chart.map(pt => pt.trucks);

                if (!historyChart) {
                    const ctx = document.getElementById('historyChart').getContext('2d');
                    historyChart = new Chart(ctx, {
                        type: 'line',
                        data: {
                            labels: labels,
                            datasets: [
                                {
                                    label: 'אנשים',
                                    data: peopleData,
                                    fill: false,
                                    borderWidth: 2
                                },
                                {
                                    label: 'רכבים',
                                    data: carsData,
                                    fill: false,
                                    borderWidth: 2
                                },
                                {
                                    label: 'משאיות',
                                    data: trucksData,
                                    fill: false,
                                    borderWidth: 2
                                }
                            ]
                        },
                        options: {
                            responsive: true,
                            plugins: {
                                legend: { display: true, position: 'bottom' }
                            },
                            scales: {
                                y: {
                                    beginAtZero: true
                                }
                            }
                        }
                    });
                } else {
                    historyChart.data.labels = labels;
                    historyChart.data.datasets[0].data = peopleData;
                    historyChart.data.datasets[1].data = carsData;
                    historyChart.data.datasets[2].data = trucksData;
                    historyChart.update();
                }
            });
        }

        setInterval(updateStats, 10000);
        window.onload = function() {
            document.getElementById('confValue').innerText = document.getElementById('conf').value;
            document.getElementById('fpsValue').innerText = document.getElementById('fps').value;
            updateStats();
        };
    </script>
    <footer style="margin-top:40px; padding:18px 0 6px 0; text-align:center; background:none; color:#777; font-size:1.07em;">
      Coded By: Ran Tenenbaum |
      <a href="https://aidevelopments.co.il/" target="_blank" style="color:#1568b2; text-decoration:none;">
        Ai Developments
      </a>
    </footer>
</body>
</html>
"""

with open("index.html", "w", encoding="utf-8") as f:
    f.write(html_code)


In [3]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 72.0 MB/s eta 0:00:00


In [4]:
!pip install ultralytics

In [5]:
!pip install pyngrok

In [6]:
!pip install torch torchvision

In [7]:
!pip install yt-dlp

In [8]:
import yt_dlp

def get_best_stream_url(youtube_url):
    ydl_opts = {'quiet': True, 'format': 'best'}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=False)
        return info['url']

YOUTUBE_STREAM_URL = 'https://www.youtube.com/watch?v=rnXIjl_Rzy4'
REAL_STREAM_URL = get_best_stream_url(YOUTUBE_STREAM_URL)
print(REAL_STREAM_URL)


https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1754318533/ei/ZXKQaN3UNNuYsfIPhp6DkQw/ip/34.125.5.150/id/rnXIjl_Rzy4.1/itag/96/source/yt_live_broadcast/requiressl/yes/ratebypass/yes/live/1/sgoap/gir%3Dyes%3Bitag%3D140/sgovp/gir%3Dyes%3Bitag%3D137/rqh/1/hls_chunk_host/rr4---sn-2oa5n5-55.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/playlist_duration/30/manifest_duration/30/bui/AY1jyLNOg4cOYNDdJKm_2wsAqaEWq6yLBS7fQcTszH5KpGlU31AnTLMITOm6qS-oRO1KhooSb6Pvk22m/spc/l3OVKbtd8YuIGffk3t3uQa3S/vprv/1/playlist_type/DVR/met/1754296935,/mh/K2/mm/44/mn/sn-2oa5n5-55/ms/lva/mv/u/mvi/4/pl/20/rms/lva,lva/dover/11/pacing/0/keepalive/yes/fexp/51355912/mt/1754295374/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgoap,sgovp,rqh,xpc,playlist_duration,manifest_duration,bui,spc,vprv,playlist_type/sig/AJfQdSswRgIhAMMYNjzAYAEj4lWnU7QMj864lPG0MpXz9YniGFUvodOhAiEA5sBeS3jl67BrMSVbV-SpwoqrKVM0ZvZizqSqX3HpbH0%3D/lsparams/hls_chunk_host,met,mh,mm,mn,ms,mv,mvi,pl,rms/lsig/APaTxxMwRQIhAIxMOGPGB

In [9]:
pip install pillow


In [10]:
!pip install ultralytics

In [1]:
from pyngrok import ngrok
ngrok.set_auth_token("30hcDgUttzwFLvyIgD822xjLxXy_5Pc4P4KRojCHPy1MnNAFe")  # תוכל לקבל מ-ngrok.com
public_url = ngrok.connect(8000)
print("Public URL:", public_url)


ModuleNotFoundError: No module named 'pyngrok'

In [12]:
import os
import cv2
import numpy as np
import subprocess
from ultralytics import YOLO
from flask import Flask, Response, render_template, request, session, redirect, url_for, jsonify
import yt_dlp
from collections import deque
import threading
import time

app = Flask(__name__)
app.secret_key = 'yolo-people-counter'

model = YOLO('yolov8x.pt')

# Ensure the templates directory exists
os.makedirs('templates', exist_ok=True)

# Write the HTML content to a file named index.html in the templates directory
html_content = """
<!DOCTYPE html>
<html lang=\"he\" dir=\"rtl\">
<head>
    <meta charset=\"utf-8\">
    <title>People Counter - AI Developments</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 40px; background: #fafbfc; }
        .container { display: flex; flex-direction: row; justify-content: center; align-items: flex-start; gap: 40px; }
        .video-box { flex: 2; }
        .params-box {
            flex: 1;
            background: #fff;
            border-radius: 12px;
            box-shadow: 0 2px 8px #0001;
            padding: 32px 24px;
            min-width: 320px;
            max-width: 400px;
        }
        .params-box h3 { margin-top: 0; }
        label { font-weight: bold; }
        .param-group { margin-bottom: 20px; }
        .param-group input[type=\"text\"], .param-group input[type=\"number\"] { width: 100%; padding: 5px; border: 1px solid #bbb; border-radius: 5px; }
        .param-group input[type=\"range\"] { width: 85%; }
        .param-group input[type=\"checkbox\"] { margin-left: 8px; }
        .save-btn {
            background: #1b83e3;
            color: white;
            border: none;
            border-radius: 7px;
            padding: 10px 30px;
            font-size: 1.2em;
            cursor: pointer;
        }
        .save-btn:hover { background: #1568b2; }
        #stats-box {
            margin-top: 40px;
            background: #f4f6f9;
            border-radius: 12px;
            box-shadow: 0 2px 8px #0001;
            padding: 24px;
            direction: rtl;
        }
        #stats-box hr { margin: 16px 0; }
        #historyChart { background: #fff; border-radius: 8px; }
    </style>
</head>
<body>
    <h1>AI Developments - מונה אנשים בבינה מלאכותית</h1>
    <div class=\"container\">
        <!-- חלק שמאל: וידאו -->
        <div class=\"video-box\">
            <img id=\"videoStream\" src=\"/video_feed\" width=\"850\" style=\"border-radius: 12px; box-shadow: 0 4px 24px #0002;\">
            <!-- סטטיסטיקות מתעדכנות -->
            <div id=\"stats-box\">
                <h3>סטטיסטיקות בזמן אמת (10 דקות אחרונות)</h3>
                <div>
                    <strong>ממוצע אנשים:</strong> <span id=\"avg_people\">-</span><br>
                    <strong>ממוצע רכבים:</strong> <span id=\"avg_cars\">-</span><br>
                    <strong>ממוצע משאיות:</strong> <span id=\"avg_trucks\">-</span>
                </div>
                <hr>
                <h3>גרף שעה אחרונה</h3>
                <canvas id=\"historyChart\" height=\"130\"></canvas>
            </div>
        </div>
        <!-- חלק ימין: פרמטרים -->
        <div class=\"params-box\">
            <form method=\"post\">
                <h3>הגדרות זיהוי</h3>
                <!-- כתובת YouTube -->
                <div class=\"param-group\">
                    <label for=\"youtube_url\">כתובת YouTube Live:</label><br>
                    <input type=\"text\" id=\"youtube_url\" name=\"youtube_url\" value=\"{{ youtube_url|e }}\">
                </div>
                <!-- Drop down -->
                <div class=\"param-group\">
                    <label for=\"predefined_stream\">או בחר ממצלמות לייב פופולריות:</label><br>
                    <select id=\"predefined_stream\">
                        <option value=\"\">-- בחר ממצלמות --</option>
                        {% for stream in predefined_streams %}
                            <option value=\"{{ stream.url }}\">{{ stream.name }}</option>
                        {% endfor %}
                    </select>
                </div>
                <!-- סליידר קונפידנס -->
                <div class=\"param-group\">
                    <label for=\"conf\">סף זיהוי (Confidence):</label><br>
                    <input type=\"range\" id=\"conf\" name=\"conf\" min=\"0.01\" max=\"1\" step=\"0.01\" value=\"{{ conf }}\">
                    <span id=\"confValue\">{{ conf }}</span>
                </div>
                <!-- צ'קבוקסים לאובייקטים -->
                <div class=\"param-group\">
                    <label>מה לזהות?</label><br>
                    <input type=\"checkbox\" id=\"detect_people\" name=\"detect_people\" {% if detect_people %}checked{% endif %}>
                    <label for=\"detect_people\">אנשים</label>
                    <input type=\"checkbox\" id=\"detect_cars\" name=\"detect_cars\" {% if detect_cars %}checked{% endif %}>
                    <label for=\"detect_cars\">מכוניות</label>
                    <input type=\"checkbox\" id=\"detect_trucks\" name=\"detect_trucks\" {% if detect_trucks %}checked{% endif %}>
                    <label for=\"detect_trucks\">משאיות</label>
                </div>
                <!-- סליידר פריימים בשנייה -->
                <div class=\"param-group\">
                    <label for=\"fps\">פריימים לשנייה:</label><br>
                    <input type=\"range\" id=\"fps\" name=\"fps\" min=\"1\" max=\"10\" step=\"1\" value=\"{{ fps }}\">
                    <span id=\"fpsValue\">{{ fps }}</span>
                </div>
                <div class=\"param-group\" style=\"text-align: left;\">
                    <button type=\"submit\" class=\"save-btn\">שמור</button>
                </div>
            </form>
        </div>
    </div>
    <!-- צ'ארט גרף -->
    <script src=\"https://cdn.jsdelivr.net/npm/chart.js\"></script>
    <script>
        // עדכון ערך קונפידנס לייד הסליידר
        document.getElementById('conf').oninput = function() {
            document.getElementById('confValue').innerText = this.value;
        };
        document.getElementById('fps').oninput = function() {
            document.getElementById('fpsValue').innerText = this.value;
        };
        // כאשר נבחר סטרים דרופדאון - מלא אוטומטית באינפוט
        document.getElementById('predefined_stream').onchange = function() {
            if (this.value) {
                document.getElementById('youtube_url').value = this.value;
            }
        };
        // סטטיסטיקות וגרף
        let historyChart = null;

        function updateStats() {
            fetch('/stats')
            .then(r => r.json())
            .then(data => {
                document.getElementById('avg_people').innerText = data.avg_people_10min;
                document.getElementById('avg_cars').innerText = data.avg_cars_10min;
                document.getElementById('avg_trucks').innerText = data.avg_trucks_10min;

                const labels = data.chart.map(pt => {
                    const date = new Date(pt.t * 1000);
                    return date.getHours().toString().padStart(2, '0') + ':' + date.getMinutes().toString().padStart(2, '0');
                });
                const peopleData = data.chart.map(pt => pt.people);
                const carsData = data.chart.map(pt => pt.cars);
                const trucksData = data.chart.map(pt => pt.trucks);

                if (!historyChart) {
                    const ctx = document.getElementById('historyChart').getContext('2d');
                    historyChart = new Chart(ctx, {
                        type: 'line',
                        data: {
                            labels: labels,
                            datasets: [
                                {
                                    label: 'אנשים',
                                    data: peopleData,
                                    fill: false,
                                    borderWidth: 2
                                },
                                {
                                    label: 'רכבים',
                                    data: carsData,
                                    fill: false,
                                    borderWidth: 2
                                },
                                {
                                    label: 'משאיות',
                                    data: trucksData,
                                    fill: false,
                                    borderWidth: 2
                                }
                            ]
                        },
                        options: {
                            responsive: true,
                            plugins: {
                                legend: { display: true, position: 'bottom' }
                            },
                            scales: {
                                y: {
                                    beginAtZero: true
                                }
                            }
                        }
                    });
                } else {
                    historyChart.data.labels = labels;
                    historyChart.data.datasets[0].data = peopleData;
                    historyChart.data.datasets[1].data = carsData;
                    historyChart.data.datasets[2].data = trucksData;
                    historyChart.update();
                }
            });
        }

        setInterval(updateStats, 10000);
        window.onload = function() {
            document.getElementById('confValue').innerText = document.getElementById('conf').value;
            document.getElementById('fpsValue').innerText = document.getElementById('fps').value;
            updateStats();
        };
    </script>
</body>
</html>
"""

with open('templates/index.html', 'w', encoding='utf-8') as f:
    f.write(html_content)


# רשימת סטרימים
PREDEFINED_STREAMS = [
    {"name": "New York - time square", "url": "https://www.youtube.com/watch?v=rnXIjl_Rzy4"},
    {"name": "New Orleans - street view", "url": "https://www.youtube.com/watch?v=qHW8srS0ylo"},
    {"name": "Dublin Ireland - main street", "url": "https://www.youtube.com/watch?v=u4UZ4UvZXrg"},
    {"name": "Jerusalem Israel - Western wall", "url": "https://www.youtube.com/watch?v=QhHYxiUlf-o"},
    {"name": "Jerusalem Israel - Western wall floor", "url": "https://www.youtube.com/watch?v=tg2ceBQnYKc"},
    {"name": "London UK - Abbey Road", "url": "https://www.youtube.com/watch?v=57w2gYXjRic"},
    {"name": "Kho Samui - Street Bars ", "url": "https://www.youtube.com/watch?v=DwKCna1mumk"},
    {"name": "Sturgis USA - Harly Davidson ", "url": "https://www.youtube.com/watch?v=uu82CI54eJU"},
    {"name": "Sukhumvit Road | Bangkok  ", "url": "https://www.youtube.com/watch?v=UemFRPrl1hk"},
    {"name": "Hollywood Beach - USA  ", "url": "https://www.youtube.com/watch?v=cmkAbDUEoyA"},
    {"name": "Alberta - Canada ", "url": "https://www.youtube.com/watch?v=_0wPODlF9wU"},
    {"name": "Supercar Spotting - California", "url": "https://www.youtube.com/watch?v=qMYlpMsWsBE"},
    {"name": "Virgin Island - Beach Bar ", "url": "https://www.youtube.com/watch?v=2wqpy036z24"},
    {"name": "Carebean - Beach", "url": "https://www.youtube.com/watch?v=loHbMM9JfCs"},
    {"name": "Manila Philpines - Resturant", "url": "https://www.youtube.com/watch?v=2sHUoP0WqGk"},
    {"name": "Melborn Australia - Bar", "url": "https://www.youtube.com/watch?v=w47yvCftkWQ"},
    {"name": "Pattya - Beach Road", "url": "https://www.youtube.com/watch?v=g_EiknClhwg"},
    {"name": "Africa  - Elefents", "url": "https://www.youtube.com/watch?v=XsOU8JnEpNM"},
    {"name": "Africa - Zebra", "url": "https://www.youtube.com/watch?v=RZFMYVNhd4k"},
    {"name": "Melborn Australia - Bar", "url": "https://www.youtube.com/watch?v=w47yvCftkWQ"},
    {"name": "Camelthorn Waterhole  - Wild", "url": "https://www.youtube.com/watch?v=mk02pEy3FdA"},
    {"name": "Africa Namibia - Desert", "url": "https://www.youtube.com/watch?v=ydYDqZQpim8"},



]

DEFAULT_YOUTUBE_STREAM_URL = PREDEFINED_STREAMS[0]["url"]

width, height = 1280, 720

# ------------------------ סטטיסטיקות ------------------------
stats = {
    'people': deque(maxlen=60),    # 10 דקות, כל 10 שניות
    'cars': deque(maxlen=60),
    'trucks': deque(maxlen=60),
    'history': deque(maxlen=360)   # שעה, כל 10 שניות
}
stats_lock = threading.Lock()
# -------------------------------------------------------------

def get_best_stream_url(youtube_url):
    ydl_opts = {'quiet': True, 'format': 'best[height<=720]'}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=False)
        return info['url']

def start_ffmpeg_proc(real_stream_url, fps, width, height):
    ffmpeg_cmd = [
        'ffmpeg',
        '-i', real_stream_url,
        '-loglevel', 'error',
        '-f', 'image2pipe',
        f'-vf', f'fps={fps},scale={width}:{height}',
        '-vcodec', 'rawvideo',
        '-pix_fmt', 'bgr24',
        '-an', '-sn', '-dn', '-',
    ]
    return subprocess.Popen(ffmpeg_cmd, stdout=subprocess.PIPE, bufsize=10**8)

def gen_frames(conf, youtube_url, classes_list, fps):
    real_stream_url = get_best_stream_url(youtube_url)
    proc = start_ffmpeg_proc(real_stream_url, fps, width, height)
    stats_interval = 10  # seconds
    last_update = time.time()

    try:
        while True:
            raw = proc.stdout.read(width * height * 3)
            if not raw:
                continue
            frame = np.frombuffer(raw, np.uint8).reshape((height, width, 3))
            results = model(frame, classes=classes_list, conf=conf)
            boxes = results[0].boxes

            people_count = sum(1 for box in boxes if int(box.cls[0]) == 0) if 0 in classes_list else 0
            car_count = sum(1 for box in boxes if int(box.cls[0]) == 2) if 2 in classes_list else 0
            truck_count = sum(1 for box in boxes if int(box.cls[0]) == 7) if 7 in classes_list else 0

            annotated = results[0].plot(labels=False)
            y = 40
            if 0 in classes_list:
                cv2.putText(annotated, f'people: {people_count} (Conf: {conf})', (10, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                y += 40
            if 2 in classes_list:
                cv2.putText(annotated, f'cars: {car_count}', (10, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                y += 40
            if 7 in classes_list:
                cv2.putText(annotated, f'trucks: {truck_count}', (10, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

            # עדכון סטטיסטיקות כל 10 שניות
            now = time.time()
            if now - last_update >= stats_interval:
                with stats_lock:
                    stats['people'].append(people_count)
                    stats['cars'].append(car_count)
                    stats['trucks'].append(truck_count)
                    stats['history'].append({
                        'timestamp': now,
                        'people': people_count,
                        'cars': car_count,
                        'trucks': truck_count
                    })
                last_update = now

            ret, buffer = cv2.imencode('.jpg', annotated)
            frame_bytes = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')
    finally:
        proc.terminate()

@app.route('/', methods=['GET', 'POST'])
def index():
    if 'params' not in session:
        session['params'] = {
            'youtube_url': DEFAULT_YOUTUBE_STREAM_URL,
            'conf': 0.4,
            'detect_people': True,
            'detect_cars': True,
            'detect_trucks': False,
            'fps': 5
        }

    if request.method == 'POST':
        youtube_url = request.form.get('youtube_url', DEFAULT_YOUTUBE_STREAM_URL)
        conf = float(request.form.get('conf', 0.4))
        detect_people = 'detect_people' in request.form
        detect_cars = 'detect_cars' in request.form
        detect_trucks = 'detect_trucks' in request.form
        fps = int(request.form.get('fps', 5))
        session['params'] = {
            'youtube_url': youtube_url,
            'conf': conf,
            'detect_people': detect_people,
            'detect_cars': detect_cars,
            'detect_trucks': detect_trucks,
            'fps': fps
        }
        # איפוס סטטיסטיקות
        with stats_lock:
            stats['people'].clear()
            stats['cars'].clear()
            stats['trucks'].clear()
            stats['history'].clear()
        return redirect(url_for('index'))

    return render_template('index.html', **session['params'], predefined_streams=PREDEFINED_STREAMS)

@app.route('/video_feed')
def video_feed():
    params = session.get('params', {})
    youtube_url = params.get('youtube_url', DEFAULT_YOUTUBE_STREAM_URL)
    conf = float(params.get('conf', 0.4))
    fps = int(params.get('fps', 5))
    classes_list = []
    if params.get('detect_people', True):
        classes_list.append(0)
    if params.get('detect_cars', True):
        classes_list.append(2)
    if params.get('detect_trucks', False):
        classes_list.append(7)
    return Response(gen_frames(conf, youtube_url, classes_list, fps),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

# ---------- API לסטטיסטיקות ועדכון גרף ----------
@app.route('/stats')
def get_stats():
    with stats_lock:
        avg_people = round(np.mean(stats['people']), 2) if stats['people'] else 0
        avg_cars = round(np.mean(stats['cars']), 2) if stats['cars'] else 0
        avg_trucks = round(np.mean(stats['trucks']), 2) if stats['trucks'] else 0

        history_list = list(stats['history'])
        chart = []
        for point in history_list:
            chart.append({
                't': int(point['timestamp']),
                'people': point['people'],
                'cars': point['cars'],
                'trucks': point['trucks']
            })
    return jsonify({
        'avg_people_10min': avg_people,
        'avg_cars_10min': avg_cars,
        'avg_trucks_10min': avg_trucks,
        'chart': chart
    })
# -------------------------------------------------

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Aug/2025 08:43:06] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Aug/2025 08:43:06] "GET /favicon.ico HTTP/1.1" 404 -


INFO:werkzeug:127.0.0.1 - - [04/Aug/2025 08:43:16] "GET /stats HTTP/1.1" 200 -


0: 384x640 3 persons, 3 cars, 69.8ms
Speed: 25.2ms preprocess, 69.8ms inference, 449.1ms postprocess per image at shape (1, 3, 384, 640)


...


0: 384x640 13 persons, 29.0ms
Speed: 2.6ms preprocess, 29.0ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 37.3ms
Speed: 2.2ms preprocess, 37.3ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 33.0ms
Speed: 2.3ms preprocess, 33.0ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 33.4ms
Speed: 2.3ms preprocess, 33.4ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 34.1ms
Speed: 4.0ms preprocess, 34.1ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 33.5ms
Speed: 4.0ms preprocess, 33.5ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 33.2ms
Speed: 3.4ms preprocess, 33.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 31.4ms
Speed: 2.4ms preprocess, 31.4ms inference, 2.2ms postprocess per image at

INFO:werkzeug:127.0.0.1 - - [04/Aug/2025 08:50:47] "GET /stats HTTP/1.1" 200 -



0: 384x640 9 persons, 28.6ms
Speed: 3.4ms preprocess, 28.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 33.7ms
Speed: 2.4ms preprocess, 33.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 34.4ms
Speed: 5.5ms preprocess, 34.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 33.9ms
Speed: 7.2ms preprocess, 33.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 33.1ms
Speed: 7.4ms preprocess, 33.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 32.3ms
Speed: 5.5ms preprocess, 32.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 39.4ms
Speed: 2.2ms preprocess, 39.4ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 31.7ms
Speed: 2.3ms preprocess, 31.7ms inference, 1.9ms postprocess per image at sha

INFO:werkzeug:127.0.0.1 - - [04/Aug/2025 08:51:07] "GET /stats HTTP/1.1" 200 -



0: 384x640 5 persons, 41.7ms
Speed: 2.4ms preprocess, 41.7ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 32.1ms
Speed: 2.6ms preprocess, 32.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 33.1ms
Speed: 2.0ms preprocess, 33.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 33.7ms
Speed: 2.9ms preprocess, 33.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 32.7ms
Speed: 8.7ms preprocess, 32.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 32.0ms
Speed: 2.2ms preprocess, 32.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 33.6ms
Speed: 2.3ms preprocess, 33.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 29.7ms
Speed: 2.6ms preprocess, 29.7ms inference, 1.8ms postprocess per image at shape (